In [114]:
import numpy as np
import pandas as pd
from fuzzywuzzy import process

In [115]:
ratings = pd.read_csv('../data/reduced/ratings_m10.csv')
ratings.reindex()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
81111,610,159093,3.0,1493847704
81112,610,164179,5.0,1493845631
81113,610,166528,4.0,1493879365
81114,610,168250,5.0,1494273047


In [116]:
movies = pd.read_csv('../data/reduced/movies_m10_rich_pre.csv', index_col='movieId')
movies.sample()

,tmdbId,imdbId,cast,director,keywords,overview,title,genres,year
movieId,,,,,,,,,
515,1245,107943,"['Anthony Hopkins', 'Emma Thompson', 'James Fo...",James Ivory,"['newspaper', 'butler', 'country house', 'loya...",A rule bound head butler's world of manners an...,"Remains of the Day, The","['Drama', 'Romance']",1993


In [117]:
movies_title = movies[['title']]

In [118]:
rating_movie = ratings.merge(movies_title, on='movieId')
rating_movie.head()

,userId,movieId,rating,timestamp,title
0,1,1,4.0,964982703,Toy Story
1,5,1,4.0,847434962,Toy Story
2,7,1,4.5,1106635946,Toy Story
3,15,1,2.5,1510577970,Toy Story
4,17,1,4.5,1305696483,Toy Story


In [119]:
rating_movie.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75043 entries, 0 to 75042
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   userId     75043 non-null  int64  
 1   movieId    75043 non-null  int64  
 2   rating     75043 non-null  float64
 3   timestamp  75043 non-null  int64  
 4   title      75043 non-null  object 
dtypes: float64(1), int64(3), object(1)
memory usage: 2.9+ MB


In [120]:
user_movie_mat = rating_movie.pivot(index='movieId', columns='userId', values='rating').fillna(0)

In [121]:
user_movie_mat

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,...,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,...,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
6,4.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,3.0,4.0,3.0,0.0,0.0,0.0,0.0,0.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139385,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.5
139644,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.5
140110,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [122]:
def corr_between_users(user_movie_mat: pd.DataFrame, user1: str, user2: str):
    rated_movies_by_both = user_movie_mat[[user1, user2]].dropna(axis=0).values
    user1_ratings = rated_movies_by_both[:, 0]
    user2_ratings = rated_movies_by_both[:, 1]
    return np.corrcoef(user1_ratings, user2_ratings)[0,1]

In [123]:
users_list = list(user_movie_mat.columns)
movies_list = list(user_movie_mat.index)

#users_similarity_mat = np.array([[corr_between_users(user_movie_mat, user1, user2) for user1 in users_list] for user2 in users_list])
#users_similarity_mat = pd.DataFrame(users_similarity_mat, index=users_list, columns=users_list)
users_similarity_mat = pd.read_pickle('users_similarity_mat.pkl')

In [124]:
def get_rated_user_for_a_movie(user_movie_mat: pd.DataFrame, movie: int):
    return user_movie_mat.loc[movie, :].dropna().index.values


def get_top_neighbors(
    users_similarity_mat: pd.DataFrame, user: str, rated_users: str, n_neighbors: int
):
    return users_similarity_mat[user][rated_users].nlargest(n_neighbors).to_dict()


def subtract_bias(rating: float, mean_rating: float):
    return rating - mean_rating


def get_neighbor_rating_without_bias_per_movie(
    user_movie_mat: pd.DataFrame, user: str, movie: int
):
    """Substract the rating of a user from the mean rating of that user to eliminate bias"""
    mean_rating = user_movie_mat[user].mean()
    rating = user_movie_mat.loc[movie, user]
    return subtract_bias(rating, mean_rating)


def get_ratings_of_neighbors(user_movie_mat: pd.DataFrame, neighbors: list, movie: int):
    """Get the ratings of all neighbors after adjusting for biases"""
    return [
        get_neighbor_rating_without_bias_per_movie(user_movie_mat, neighbor, movie)
        for neighbor in neighbors
    ]


def get_weighted_average_rating_of_neighbors(ratings: list, neighbor_distance: list):
    weighted_sum = np.array(ratings).dot(np.array(neighbor_distance))
    abs_neigbor_distance = np.abs(neighbor_distance)
    return weighted_sum / np.sum(abs_neigbor_distance)


def ger_user_rating(user_movie_mat: pd.DataFrame, user: str, avg_neighbor_rating: float):
    user_avg_rating = user_movie_mat[user].mean()
    return round(user_avg_rating + avg_neighbor_rating, 2)

In [125]:
def predict_rating(
    df: pd.DataFrame,
    users_similarity_mat: pd.DataFrame,
    user: int,
    movie: int,
    n_neighbors: int = 2,
):
    """Predict the rating of a user for a movie based on the ratings of neighbors"""
    user_movie_mat = df.copy()

    rated_users = get_rated_user_for_a_movie(user_movie_mat, movie)

    top_neighbors_distance = get_top_neighbors(
        users_similarity_mat, user, rated_users, n_neighbors
    )
    neighbors, distance = top_neighbors_distance.keys(), top_neighbors_distance.values()

    #print(f"Top {n_neighbors} neighbors of user {user}, {movie}: {list(neighbors)}")

    ratings = get_ratings_of_neighbors(user_movie_mat, neighbors, movie)
    avg_neighbor_rating = get_weighted_average_rating_of_neighbors(
        ratings, list(distance)
    )

    return ger_user_rating(user_movie_mat, user, avg_neighbor_rating)


In [126]:
movie_name = 'toy story'
user1 = 1
movie = process.extractOne(movie_name, movies['title'])[2]
rating = predict_rating(user_movie_mat, users_similarity_mat, user1, movie)
rating

3.52

In [127]:
full_ratings = pd.read_csv('full_ratings_comp.csv', index_col='movieId')

In [128]:
def get_n_recommendations(user: str, n: int, full_ratings: pd.DataFrame, movies: pd.DataFrame):
    user_ratings = full_ratings[[user]]
    user_ratings.sort_values(by=user, ascending=False, inplace=True)
    recommendations = user_ratings.iloc[0:n].index
    recommended_movies = pd.DataFrame(columns=movies.columns)
    for r in recommendations:
        recommended_movies = pd.concat([recommended_movies, movies[movies.index == r]])
    return recommended_movies

In [129]:
get_n_recommendations('1', 10, full_ratings, movies)

C:\Users\felma\AppData\Local\Temp\ipykernel_2652\1905796414.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_ratings.sort_values(by=user, ascending=False, inplace=True)


,tmdbId,imdbId,cast,director,keywords,overview,title,genres,year
2529,871,63442,"['Charlton Heston', 'Roddy McDowall', 'Kim Hun...",Franklin J. Schaffner,"['human evolution', 'gorilla', 'bondage', 'spa...",Taylor and two other astronauts come out of de...,Planet of the Apes,"['Action', 'Drama', 'Sci-Fi']",1968
2596,6396,133189,"['Matthew Lillard', 'Michael A. Goorjian', 'An...",James Merendino,"['punk', 'independent film']","Two former geeks become 1980s punks, then part...",SLC Punk!,"['Comedy', 'Drama']",1998
3578,98,172495,"['Russell Crowe', 'Joaquin Phoenix', 'Connie N...",Ridley Scott,"['rome', 'gladiator', 'roman empire', 'slavery...",General Maximus' success in battle earns the f...,Gladiator,"['Action', 'Adventure', 'Drama']",2000
231,8467,109686,"['Jim Carrey', 'Jeff Daniels', 'Lauren Holly',...",Peter Farrelly,"['gas station', 'motel', 'utah', 'stupidity', ...",Lloyd and Harry are two men whose stupidity is...,Dumb & Dumber (Dumb and Dumber),"['Adventure', 'Comedy']",1994
2542,100,120735,"['Jason Flemyng', 'Dexter Fletcher', 'Nick Mor...",Guy Ritchie,"['ambush', 'alcohol', 'shotgun', 'tea', 'joint']",A card sharp and his unwillingly-enlisted frie...,"Lock, Stock & Two Smoking Barrels","['Comedy', 'Crime', 'Thriller']",1998
2571,603,133093,"['Keanu Reeves', 'Laurence Fishburne', 'Carrie...",Lilly Wachowski|Lana Wachowski,"['saving the world', 'artificial intelligence'...","Set in the 22nd century, The Matrix tells the ...","Matrix, The","['Action', 'Sci-Fi', 'Thriller']",1999
2580,9430,139239,"['Sarah Polley', 'Katie Holmes', 'Jay Mohr', '...",Doug Liman,"['ecstasy', 'drug dealing', 'drug', 'reference...","Told from three perspectives, a story of a bun...",Go,"['Comedy', 'Crime']",1999
1240,218,88247,"['Arnold Schwarzenegger', 'Michael Biehn', 'Li...",James Cameron,"['saving the world', 'artificial intelligence'...","In the post-apocalyptic future, reigning tyran...","Terminator, The","['Action', 'Sci-Fi', 'Thriller']",1984
661,10539,116683,"['Miriam Margolyes', 'Joanna Lumley', 'Pete Po...",Henry Selick,"['based on novel', 'garden', 'loss of parents'...",When the young orphan boy James spills a magic...,James and the Giant Peach,"['Adventure', 'Animation', 'Children', 'Fantas...",1996
3033,957,94012,"['Mel Brooks', 'Rick Moranis', 'Bill Pullman',...",Mel Brooks,"['android', 'lasergun', 'swordplay', 'temple',...",When the nefarious Dark Helmet hatches a plan ...,Spaceballs,"['Comedy', 'Sci-Fi']",1987


In [131]:
def ger_full_ratings():
    full_ratings = user_movie_mat.copy()

    for user, movies in full_ratings.items():
        for movie in movies.keys():
            if np.isnan(full_ratings.loc[movie, user]) or full_ratings.loc[movie, user] == 0:
                full_ratings.loc[movie, user] = predict_rating(
                    user_movie_mat, users_similarity_mat, user, movie
                )

In [132]:
full_ratings

,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.00,-0.0,-0.0,0.78,4.00,0.05,4.50,0.01,0.01,0.02,...,4.00,0.03,4.00,3.0,4.00,2.50,4.00,2.50,3.00,5.00
2,0.03,-0.0,-0.0,-0.13,1.04,4.00,0.04,4.00,0.01,0.02,...,-0.03,4.00,0.04,5.0,3.50,0.65,0.87,2.00,-0.01,1.23
3,4.00,-0.0,-0.0,-0.13,1.04,5.00,0.04,0.01,0.01,0.02,...,-0.03,0.03,0.04,0.0,0.05,-0.17,-0.00,2.00,-0.01,-0.02
5,0.03,-0.0,-0.0,-0.13,1.04,5.00,0.04,0.01,0.01,0.02,...,-0.03,0.03,0.04,3.0,0.05,0.24,-0.00,0.19,-0.01,-0.02
6,4.00,-0.0,-0.0,0.78,1.04,4.00,0.04,0.01,0.01,0.02,...,-0.03,3.00,4.00,3.0,0.05,0.65,-0.00,1.39,-0.01,5.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139385,0.03,-0.0,-0.0,-0.13,-0.02,0.05,0.04,0.01,0.01,0.02,...,1.79,0.03,0.04,0.0,0.05,-0.17,-0.00,0.19,-0.01,4.50
139644,0.03,-0.0,-0.0,-0.13,-0.02,0.05,0.04,0.01,0.01,0.02,...,-0.03,0.03,0.04,0.0,0.05,-0.17,-0.00,0.19,-0.01,4.50
140110,0.03,-0.0,-0.0,-0.13,-0.02,0.05,0.04,0.01,0.01,5.00,...,-0.03,0.03,0.04,0.0,0.05,-0.17,-0.00,0.19,-0.01,-0.02


In [133]:
user_movie_mat.to_csv('user_movie_mat.csv')
users_similarity_mat.to_pickle('users_similarity_mat.pkl')